# Load profiles and saev it as a parquet file

## Import libraries

In [1]:
import io
import pandas as pd
import plotly.express as px
import plotly.io as pio

## Helper functions

In [2]:
profile_formatter = (
    "s3://cellpainting-gallery/cpg0016-jump/"
    "{Metadata_Source}/workspace/profiles/"
    "{Metadata_Batch}/{Metadata_Plate}/{Metadata_Plate}.parquet"
)

## Load metadata

In [3]:
plates = pd.read_csv("../../datasets/metadata/plate.csv.gz")
wells = pd.read_csv("../../datasets/metadata/well.csv.gz")
compound = pd.read_csv("../../datasets/metadata/compound.csv.gz")
orf = pd.read_csv("../../datasets/metadata/orf.csv.gz")


## Load plates

In [11]:
sample = (
    plates.query('Metadata_Source=="source_4"')
    .query('Metadata_PlateType=="ORF"')
    .query('Metadata_Batch=="2021_07_12_Batch8" | Metadata_Batch=="2021_07_26_Batch9"') # drop this later
    .sample(2)
)

# count the number of plates per batch
sample.groupby("Metadata_Batch")["Metadata_Plate"].count()


Metadata_Batch
2021_07_12_Batch8    1
2021_07_26_Batch9    1
Name: Metadata_Plate, dtype: int64

## Loading profiles

Now let's load the profiles from these plates.

Setting `columns = None` below will load all of the features.

<div class="alert alert-warning">
WARNING: Files are located in S3. This loop loads only two features per each sampled plate; loading many feature and/or many plates can take several minutes.
</div>

In [13]:
dframes = []
columns = [
    "Metadata_Source",
    "Metadata_Plate",
    "Metadata_Well",
    "Cells_AreaShape_Eccentricity",
    "Nuclei_AreaShape_Area",
]
for _, row in sample.iterrows():
    s3_path = profile_formatter.format(**row.to_dict())
    dframes.append(
        pd.read_parquet(s3_path, storage_options={"anon": True}, columns=columns)
    )
dframes = pd.concat(dframes)

ImportError: Missing optional dependency 'fsspec'.  Use pip or conda to install fsspec.